# AIPI 590 - XAI | Adversarial Patches
### Description
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/main/adversarial_patch.ipynb)